# Introduction

Some example code showing how to automatically download files from a Google drive folder, then load all csv and excel into a Postgres database. The code will handle csv encoding issues as well as load all tabs in each Excel file. 

However, it is only an example and would need more work because:

- Some Excel files have more than one table on a sheet
- Many Excel files have blank rows, you'd need to scan through them with openpyxl and remove so column heading are identified
- Table names are cleaned <FILE NAME>_<SHEET NAME IF EXCEL WITH MULTIPLE SHEETS>

The examples chosen are for the Save the Children stream of the DataKind 2022 DataDive.

- [project brief](https://docs.google.com/document/d/1TQ2TiGK_k8KEIUPzVb3ZSKaxzEBP9s9ZJBI2U_HeQ6U/edit#)
- [Google drive with data](https://drive.google.com/drive/folders/1G_CAhpb0xV9zRrV-T5ngA03x-IZ4tGKz)

The example is meant to show how we might load all data into a database to see how we might link data together, build classification and persist meta data. There are many other ways of course, but good 'ol SQL might be a good start!

## Setup

1. Go to the DataDive [google folder](https://drive.google.com/drive/folders/1eBQZhSeCXC9a62Dpg8_cWeKIZXfrxfGz)
2. Right-click on a sub-folder and Create a shortcut in your My Drive, eg for Kenya folder
3. Run pane to mount google drive below, connect your drive
5. Edit `dir` variable below and set path as seen in the laft pane

In [ ]:
# If you don't have these installed, uncomment these lines and run
#%pip install gdown
#%pip install chardet
#%pip install sqlalchemy
#%pip install psycopg2-binary

In [ ]:
import gdown
import os
import pandas as pd
import chardet

## Examples
### Downloading a folder from gdrive

In [17]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#folder_url = "https://drive.google.com/drive/folders/1G_CAhpb0xV9zRrV-T5ngA03x-IZ4tGKz?usp=share_link"
#gdown.download_folder(folder_url, quiet=True, use_cookies=False)

Mounted at /content/drive


## Reading files in a directory to pandas

This snippet will read all csv and xls files in a directory into pandas dataframes, create a list of them.

Just an example, needs a bit more logic to handle xlsx sheets which hold multiple tables, blank rows etc.

In [34]:
country = "Kenya"
dir = f"/content/drive/MyDrive/{country}"

#dir = "/content/drive/MyDrive/Kenya Cleaned Up Files"


meta_data = pd.read_csv(f"{dir}/Data metadata - Sheet1.csv")
file_col = 'Google Drive or Github link to downloaded data (if you can put what you’ve downloaded into a shared location!)'

# Array of pandas dataframes
dfs = {}

for f in os.listdir(dir):
    file = f
    f = f"{dir}/{f}"
    if '.csv' in f:
        print(f"Loading csv file {f}")
        # Detect encoding
        with open(f, 'rb') as rawdata:
            r = chardet.detect(rawdata.read(100000))
        df = pd.read_csv(f, encoding=r['encoding'])
        df['file'] = file
        dfs[f] = df
    if '.xlsx' in f:
        print(f"Leading {f} ...")
        # Read all sheets in
        sheet_to_df_map = pd.read_excel(f, sheet_name=None)
        for sheet in sheet_to_df_map:
          # TODO
          # 1. TEST FOR BLANK ROWS AT TOP, AND REMOVE
          # 2. Does the table have a blank column, if so, is it two tables on one sheet. If so handle that
          df = sheet_to_df_map[sheet]
          df['file'] = file
          dfs[f"{f} - {sheet}"] = df 


Leading /content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx ...


/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Leading /content/drive/MyDrive/Kenya/Kenya - IPC Analysis 2017-2022.xlsx ...
Loading csv file /content/drive/MyDrive/Kenya/wfp_food_prices_ken.csv
Leading /content/drive/MyDrive/Kenya/retail-prices-for-dry-beans-2017-per-kg.xlsx ...
Leading /content/drive/MyDrive/Kenya/retail-prices-for-dry-beans-2018-per-kg.xlsx ...
Leading /content/drive/MyDrive/Kenya/retail-prices-for-dry-beans-2019-per-kg.xlsx ...
Leading /content/drive/MyDrive/Kenya/value-of-recorded-marketed-agricultural-production-at-current-prices-2014-2018.xlsx ...
Leading /content/drive/MyDrive/Kenya/ken_admpop_2019.xlsx ...
Loading csv file /content/drive/MyDrive/Kenya/dbo-foodconsumptionscores.csv
Leading /content/drive/MyDrive/Kenya/per-capita-food-consumption-2017-and-2018.xlsx ...
Leading /content/drive/MyDrive/Kenya/ken_adminboundaries_tabulardata.xlsx ...
Loading csv file /content/drive/MyDrive/Kenya/ken_admpop_adm0_2019.csv
Loading csv file /content/drive/MyDrive/Kenya/ken_admpop_adm1_2019.csv
Loading csv file /conten

In [52]:
print(f"\n\nHere are the shapes of the dataframes we just loaded from directory {dir} ... \n\n")
for key in dfs:
    print("===================================================================================================\n\n")
    df = dfs[key]
    f = df['file'][0]
    print(f"File: {f} > Size: {df.shape}\n\n")
    file_meta = meta_data.loc[meta_data[file_col] == f]
    description = file_meta['Dataset description'][0]
    link = file_meta['Dataset'][0]
    #print("Link: ", str(link),"\n\n")
    print("Description: ",str(description),"\n\n")
    display(df)


print("Done")



Here are the shapes of the dataframes we just loaded from directory /content/drive/MyDrive/Kenya ... 




File: kenya-3w-final-list-2017.xlsx > Size: (1008, 6)


Description:  Kenya - Who is doing what and where 2017

Dataset shows who is doing what and where (3W) in kenya. This data was collected for the Kenya drought response 




,COUNTY,NAME,TYPE,ABBREV,SECTOR,file
0,Bungoma,United Nations Childrens Fund,UN,UNICEF,Health,kenya-3w-final-list-2017.xlsx
1,Bungoma,United Nations Childrens Fund,UN,UNICEF,Education,kenya-3w-final-list-2017.xlsx
2,Bungoma,United Nations Childrens Fund,UN,UNICEF,Nutrition,kenya-3w-final-list-2017.xlsx
3,Bungoma,United Nations Childrens Fund,UN,UNICEF,Protection,kenya-3w-final-list-2017.xlsx
4,Bungoma,United Nations Childrens Fund,UN,UNICEF,Emergency Response,kenya-3w-final-list-2017.xlsx
...,...,...,...,...,...,...
1003,Tharaka - Nithi,World Food Programme,UN,WFP,Education,kenya-3w-final-list-2017.xlsx
1004,Embu,World Food Programme,UN,WFP,Education,kenya-3w-final-list-2017.xlsx
1005,Isiolo,World Food Programme,UN,WFP,Emergency Response,kenya-3w-final-list-2017.xlsx
1006,Samburu,World Food Programme,UN,WFP,Nutrition,kenya-3w-final-list-2017.xlsx




File: kenya-3w-final-list-2017.xlsx > Size: (42, 15)


Description:  Kenya - Who is doing what and where 2017

Dataset shows who is doing what and where (3W) in kenya. This data was collected for the Kenya drought response 




,COUNTY,COUNTY.1,DRR,Education,Emergency,Food,Governance,Health,logistics,Nutrition,Protection,Shelter,WaSH,Total,file
0,Baringo,BARINGO,1.0,2.0,1.0,1.0,NaN,1.0,NaN,NaN,2.0,1.0,2.0,5,kenya-3w-final-list-2017.xlsx
1,Bomet,BOMET,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2,kenya-3w-final-list-2017.xlsx
2,Bungoma,BUNGOMA,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,2,kenya-3w-final-list-2017.xlsx
3,Busia,BUSIA,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,1.0,2,kenya-3w-final-list-2017.xlsx
4,Elgeyo-Marakwet,ELGEYO-MARAKWET,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,2.0,NaN,1.0,3,kenya-3w-final-list-2017.xlsx
5,Embu,EMBU,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,kenya-3w-final-list-2017.xlsx
6,Garissa,GARISSA,1.0,3.0,3.0,3.0,1.0,18.0,NaN,NaN,2.0,18.0,20.0,24,kenya-3w-final-list-2017.xlsx
7,Homa Bay,HOMA BAY,1.0,1.0,3.0,1.0,1.0,1.0,NaN,NaN,2.0,1.0,2.0,4,kenya-3w-final-list-2017.xlsx
8,Isiolo,ISIOLO,1.0,6.0,NaN,8.0,2.0,6.0,NaN,6.0,1.0,8.0,NaN,10,kenya-3w-final-list-2017.xlsx
9,Kajiado,KAJIADO,NaN,2.0,1.0,1.0,NaN,1.0,NaN,NaN,2.0,NaN,2.0,3,kenya-3w-final-list-2017.xlsx




File: kenya-3w-final-list-2017.xlsx > Size: (42, 4)


Description:  Kenya - Who is doing what and where 2017

Dataset shows who is doing what and where (3W) in kenya. This data was collected for the Kenya drought response 




...

,Unnamed: 0,Unnamed: 1,file
0,Baringo,WVI,kenya-3w-final-list-2017.xlsx
1,Embu,Trocaire,kenya-3w-final-list-2017.xlsx
2,Garissa,ADESO,kenya-3w-final-list-2017.xlsx
3,Homa Bay,PLAN,kenya-3w-final-list-2017.xlsx
4,Isiolo,ADESO,kenya-3w-final-list-2017.xlsx
5,Kilifi,PLAN,kenya-3w-final-list-2017.xlsx
6,Kisumu,PLAN,kenya-3w-final-list-2017.xlsx
7,Kitui,Trocaire,kenya-3w-final-list-2017.xlsx
8,Kwale,PLAN,kenya-3w-final-list-2017.xlsx
9,Machakos,PLAN,kenya-3w-final-list-2017.xlsx




File: kenya-3w-final-list-2017.xlsx > Size: (44, 14)


Description:  Kenya - Who is doing what and where 2017

Dataset shows who is doing what and where (3W) in kenya. This data was collected for the Kenya drought response 




,County,DRR/DRM,Education,Emergency response,Food security and livelihoods,Governance,Health,logistics,Nutrition,Protection,Shelter and NFI,WaSH,Total,file
0,Baringo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kenya-3w-final-list-2017.xlsx
1,Bomet,WVI,NaN,KRCS,NaN,NaN,NaN,NaN,NaN,WVI,NaN,WVI,2.0,kenya-3w-final-list-2017.xlsx
2,Embu,Trocaire,NaN,NaN,Trocaire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,kenya-3w-final-list-2017.xlsx
3,Kiambu,NaN,AVSI,NaN,NaN,NaN,NaN,NaN,NaN,AVSI,NaN,NaN,1.0,kenya-3w-final-list-2017.xlsx
4,Kisumu,Plan,Plan,"KRCS, Plan",WVI,"CORDAID, Plan, WVI",NaN,NaN,NaN,"Plan, WVI",Plan,Plan,4.0,kenya-3w-final-list-2017.xlsx
5,Kwale,Plan,Plan,"KRCS, Plan",SP,Plan,NaN,NaN,NaN,Plan,Plan,Plan,3.0,kenya-3w-final-list-2017.xlsx
6,Lamu,NaN,WVI,KRCS,NaN,WVI,NaN,NaN,NaN,WVI,NaN,WVI,2.0,kenya-3w-final-list-2017.xlsx
7,Machakos,Plan,Plan,"KRCS, Plan",NaN,"WVI, Plan",NaN,NaN,NaN,Plan,Plan,Plan,3.0,kenya-3w-final-list-2017.xlsx
8,Mombasa,NaN,NaN,KRCS,NaN,WVI,NaN,NaN,NaN,NaN,NaN,SP,3.0,kenya-3w-final-list-2017.xlsx
9,Nyandarua,NaN,AVSI,KRCS,NaN,NaN,NaN,NaN,NaN,AVSI,NaN,NaN,2.0,kenya-3w-final-list-2017.xlsx




File: Kenya - IPC Analysis 2017-2022.xlsx > Size: (231, 56)




KeyError: ignored

## Loading pandas dataframes into a postgres database

Roughcode for how one might load all the dataframes into a SQL database. You can install postgres locally, or use docker and run [this](https://hub.docker.com/_/postgres).

In [ ]:
from sqlalchemy import create_engine  
import sqlalchemy

# Set these for your Postgres instance
host   = 'localhost' 
dbname = 'postgres'
user   = 'postgres'
pwd    = os.getenv['PWD'] # Don't save passwords in notebooks - that's naughty! :) - use environment variables. :)
port   = '5432'

engine = create_engine('postgresql+psycopg2://' + user + ':' + pwd + '@' + host +':' + port + '/' + dbname)

for k in dfs:
    df = dfs[k]
    table = k.replace('.','').replace(' ','_').replace('-','_').replace('/','').replace(',','')[0:62]
    print(f"Loading into database table {table} ...")
    df.to_sql(table, engine, if_exists='replace')

print("Done")

Loading into database table Kenyafts_requirements_funding_covid_kencsv ...
Loading into database table Kenyadbo_foodconsumptionscorescsv ...
Loading into database table Kenyaper_capita_food_consumption_2017_and_2018xlsx___Sheet1 ...
Loading into database table Kenyaretail_prices_for_dry_beans_2019_per_kgxlsx___Prices_for_ ...
Loading into database table Kenyavalue_of_recorded_marketed_agricultural_production_at_cur ...
Loading into database table Kenyawfp_food_prices_kencsv ...
Loading into database table Kenyafts_requirements_funding_cluster_kencsv ...
Loading into database table Kenyafts_incoming_funding_kencsv ...
Loading into database table Kenyahealth_kencsv ...
Loading into database table Kenyaken_adminboundaries_tabulardataxlsx___Admin2 ...
Loading into database table Kenyaken_adminboundaries_tabulardataxlsx___Admin1 ...
Loading into database table Kenyaken_adminboundaries_tabulardataxlsx___Admin0 ...
Loading into database table Kenyafts_requirements_funding_kencsv ...
Loading i

## Query from the database

In [ ]:
query = ' SELECT * FROM public."Kenyakenya_3w_final_list_2017xlsx___drr"'
df = pd.read_sql(query, con=engine)
display(df)

,index,Unnamed: 0,Unnamed: 1
0,0,Baringo,WVI
1,1,Embu,Trocaire
2,2,Garissa,ADESO
3,3,Homa Bay,PLAN
4,4,Isiolo,ADESO
5,5,Kilifi,PLAN
6,6,Kisumu,PLAN
7,7,Kitui,Trocaire
8,8,Kwale,PLAN
9,9,Machakos,PLAN


### Select all column names in the files which uploaded ok

In [ ]:
query = ' SELECT * FROM information_schema.columns'
df = pd.read_sql(query, con=engine)
display(df)

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,postgres,public,Kenyakenya_3w_final_list_2017xlsx___Table,Total,14,None,YES,double precision,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,postgres,pg_catalog,pg_statistic,stanumbers5,26,None,YES,ARRAY,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,postgres,pg_catalog,pg_statistic,stavalues1,27,None,YES,anyarray,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,postgres,pg_catalog,pg_statistic,stavalues2,28,None,YES,anyarray,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,postgres,pg_catalog,pg_statistic,stavalues3,29,None,YES,anyarray,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2843,postgres,information_schema,sql_parts,feature_id,1,None,YES,character varying,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2844,postgres,pg_catalog,pg_partitioned_table,partexprs,8,None,YES,pg_node_tree,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2845,postgres,information_schema,sql_parts,feature_name,2,None,YES,character varying,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2846,postgres,information_schema,sql_parts,is_supported,3,None,YES,character varying,3.0,1.200000e+01,...,NO,None,None,None,None,None,NO,NEVER,None,YES
